## Example Case with sktime data

In [1]:
# %%
from sklearn.pipeline import make_pipeline
from sktime.forecasting.compose._reduce import (
    RecursiveReductionForecaster,
)
from xgboost import XGBRegressor
from sktime.forecasting.model_selection import (
    ForecastingGridSearchCV,
    ForecastingOptunaSearchCV,
)
from sktime.performance_metrics.forecasting import MeanAbsoluteError
from sktime.utils.estimator_checks import check_estimator

import optuna

parallel_config = {
    "backend:parallel": "joblib",
    "backend:parallel:params": {"backend": "loky", "n_jobs": -1},  # deactivate parallel here
}

lags = 12

params_xgb = {
    "random_state": 42,
}

regressor = make_pipeline(
    XGBRegressor(**params_xgb),
)

model_xgb = RecursiveReductionForecaster(
    estimator=regressor,
    impute_method="bfill",
    pooling="global",
    window_length=lags,
)

pipe_forecast = model_xgb

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(


In [2]:
from utils import load_stallion

_, y = load_stallion(as_period=True)

In [3]:
# Filter the data to reduce the dataset size
y = y.reset_index()
y = y[y['agency'].str.match(r'Agency_0[1-2]$')]
y = y[y['sku'].str.match(r'SKU_0[1-3]$')]
y = y[y['date'] > '2013-12']
y = y.set_index(["agency", "sku", "date"])

In [4]:
from sktime.transformations.hierarchical.aggregate import Aggregator

agg = Aggregator(flatten_single_levels=False)
data_agg = agg.reset().fit_transform(y)

In [5]:
from sktime.forecasting.model_selection import temporal_train_test_split

y_train, y_test = temporal_train_test_split(y, test_size=18)
test_fh = y_test.index.get_level_values(-1).unique()

In [6]:
from sktime.split import ExpandingWindowSplitter, SlidingWindowSplitter
import numpy as np

parameter_search_space_optuna = {
    "estimator__xgbregressor__n_estimators": optuna.distributions.IntDistribution(10, 3000),
}

parameter_search_space_grid = {
    "estimator__xgbregressor__n_estimators": list(range(10, 3000, 10)),  # grid
}

parameter_search_space_rrf = {
    "forecaster__forecaster__estimator__xgbregressor__n_estimators": optuna.distributions.IntDistribution(10, 3000),
}

fh = 18
fold = 4
step_length=3
y_size = len(y_train.index.get_level_values(-1).unique())
single_fold_length = y_size - (fold - 1) * step_length - fh
fold_strategy =  ExpandingWindowSplitter(
                fh=np.arange(1, fh + 1), initial_window=single_fold_length, step_length=step_length
            )

sampler = optuna.samplers.TPESampler(seed=42)

grid = ForecastingGridSearchCV(
    forecaster=pipe_forecast,
    param_grid=parameter_search_space_grid,
    cv=fold_strategy,
    error_score="raise",
    scoring=MeanAbsoluteError(),
)


htcv = ForecastingOptunaSearchCV(
    forecaster=pipe_forecast,
    param_grid=parameter_search_space_optuna,
    cv=fold_strategy,
    n_evals=5,
    scoring=MeanAbsoluteError(),
    error_score="raise",
    sampler=sampler,
)

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/model_selection/_tune.py:1773: UserWarning: ForecastingOptunaSearchCV is experimental, and interfaces may change. User feedback and suggestions for future development are appreciated in issue #6618 here: https://github.com/sktime/sktime/issues/6618
  warn(


## Show Version

In [7]:
import pandas as pd
pd.show_versions()

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(



INSTALLED VERSIONS
------------------
commit           : 2e218d10984e9919f0296931d92ea851c6a6faf5
python           : 3.10.6.final.0
python-bits      : 64
OS               : Linux
OS-release       : 5.15.153.1-microsoft-standard-WSL2
Version          : #1 SMP Fri Mar 29 23:14:13 UTC 2024
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : None
LANG             : C.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.5.3
numpy            : 1.23.5
pytz             : 2024.2
dateutil         : 2.9.0.post0
setuptools       : 75.1.0
pip              : 24.2
Cython           : 3.0.11
pytest           : 7.4.2
hypothesis       : 6.112.1
sphinx           : None
blosc            : None
feather          : None
xlsxwriter       : None
lxml.etree       : None
html5lib         : None
pymysql          : None
psycopg2         : None
jinja2           : 3.1.4
IPython          : 8.27.0
pandas_datareader: None
bs4              : None
bottleneck       : None

## Check Estimator

In [ ]:
# vscode crashes
check_est_before_fit_grid = check_estimator(grid)

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(


**********************************************************************
File "/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/model_selection/_tune.py", line 552, in NoName
Failed example:
    gscv.fit(y)
Expected:
    ForecastingGridSearchCV(...)
Got:
    ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=[1, 2, 3]),
                            forecaster=NaiveForecaster(),
                            param_grid={'strategy': ['last', 'mean', 'drift']})


/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-tim

KeyboardInterrupt: 

In [ ]:
check_est_before_fit_optuna = check_estimator(htcv)

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/model_selection/_tune.py:1773: UserWarning: ForecastingOptunaSearchCV is experimental, and interfaces may change. User feedback and suggestions for future development are appreciated in issue #6618 here: https://github.com/sktime/sktime/issues/6618
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/ho

**********************************************************************
File "/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/model_selection/_tune.py", line 1697, in NoName
Failed example:
    fh = ForecastingHorizon(y_test.index, is_relative=False).to_relative(
Exception raised:
    Traceback (most recent call last):
      File "/home/l29244/.pyenv/versions/3.10.6/lib/python3.10/doctest.py", line 1350, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest NoName[18]>", line 1
        fh = ForecastingHorizon(y_test.index, is_relative=False).to_relative(
                                                                            ^
    SyntaxError: '(' was never closed
**********************************************************************
File "/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/model_selection/_tune.py", line 1698, in NoName
Failed example:
    cutoff

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/model_selection/_tune.py:1773: UserWarning: ForecastingOptunaSearchCV is experimental, and interfaces may change. User feedback and suggestions for future development are appreciated in issue #6618 here: https://github.com/sktime/sktime/issues/6618
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/ho

In [11]:
check_est_before_fit = check_estimator(model_xgb)

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-tim

FAILED: test_hierarchical_with_exogeneous[RecursiveReductionForecaster-y:1cols]
FAILED: test_hierarchical_with_exogeneous[RecursiveReductionForecaster-y:2cols]


/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(


## Fit

In [12]:
# fit works for XGBoost and RRF
only_rff_xgb_fitted = model_xgb.fit(
    y=y_train,
    fh=test_fh,
)

In [13]:
# fit fails
# TypeError: Cannot convert input [('Agency_01', 'SKU_01', Period('2013-01', 'M'))] of type <class 'tuple'> to Timestamp
optuna_fitted = htcv.fit(
    y=y_train,
    fh=test_fh,
)

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/model_selection/_tune.py:1773: UserWarning: ForecastingOptunaSearchCV is experimental, and interfaces may change. User feedback and suggestions for future development are appreciated in issue #6618 here: https://github.com/sktime/sktime/issues/6618
  warn(
[I 2024-12-06 11:48:46,754] A new study created in memory with name: no-name-33999cc5-9aa2-43c9-ac13-eaa350e488c9
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user fe

TypeError: Cannot convert input [('Agency_01', 'SKU_01', Period('2014-01', 'M'))] of type <class 'tuple'> to Timestamp

In [14]:
# fit fails
# TypeError: Cannot convert input [('Agency_01', 'SKU_01', Period('2013-01', 'M'))] of type <class 'tuple'> to Timestamp

grid_fitted = grid.fit(
    y=y_train,
    fh=test_fh,
)

/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-time-series/venv2/lib/python3.10/site-packages/sktime/forecasting/compose/_reduce.py:2325: UserWarning: RecursiveReductionForecaster is experimental, and interfaces may change. user feedback is appreciated in issue #3224 here: https://github.com/alan-turing-institute/sktime/issues/3224
  warn(
/home/l29244/churn-planning-tim

TypeError: Cannot convert input [('Agency_01', 'SKU_01', Period('2014-01', 'M'))] of type <class 'tuple'> to Timestamp

## predict

In [15]:
# fails
# TypeError: Cannot convert input [('Agency_01', 'SKU_01', Period('2013-01', 'M'))] of type <class 'tuple'> to Timestamp
only_rff_xgb_fitted.predict(fh=test_fh)

TypeError: Cannot convert input [('Agency_01', 'SKU_01', Period('2014-01', 'M'))] of type <class 'tuple'> to Timestamp

In [ ]:
optuna_fitted.predict(fh=test_fh)

volume
agency    sku    date                 
Agency_01 SKU_01 2016-07     84.921066
                 2016-08    114.410080
                 2016-09    106.359634
                 2016-10     86.365440
                 2016-11     80.918663
...                                ...
Agency_02 SKU_03 2017-08  19086.826172
                 2017-09  19086.826172
                 2017-10  19086.826172
                 2017-11  19086.826172
                 2017-12  19086.826172

[108 rows x 1 columns]

In [ ]:
grid_fitted.predict(fh=test_fh)